# Chains

LCEL : LangChain Expression Language

RunnableSequence(prompt, model, parser) = prompt | model | parser

In [1]:
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model="qwen2.5-coder:7b",
    model_provider="ollama",
    temperature = 0.0
)

## Simple Chain

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = PromptTemplate(
    template='Generate 5 interesting facts about {topic}',
    input_variables=['topic']
)

parser = StrOutputParser()

chain = prompt | model | parser

result = chain.invoke({'topic':'cricket'})

print(result)

chain.get_graph().print_ascii()

1. Cricket is the most popular sport in the world, with over three billion people watching it annually.
2. The longest one-day international match ever played lasted for 96 hours and ended in a draw between England and Australia in 2003.
3. The first-ever cricket ball was made from cork and covered with leather, while modern balls are made of synthetic materials like rubber and yarn.
4. The fastest century (100 runs) in Test cricket history was scored by Brian Lara of Trinidad and Tobago in 1987, taking just 2 hours and 5 minutes.
5. Cricket has been played in space! In 2007, astronauts from the European Space Agency played a match on the International Space Station using specially designed equipment.
     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
         

## Sequential Chain

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt1 = PromptTemplate(
    template='Generate a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate a 5 pointer summary from the following text \n {text}',
    input_variables=['text']
)

parser = StrOutputParser()

chain = prompt1 | model | parser | prompt2 | model | parser

result = chain.invoke({'topic': 'Unemployment in India'})

print(result)

chain.get_graph().print_ascii()

1. **High Unemployment Rates:** As of 2023, India's official unemployment rate stands at around 5.8%, with youth unemployment rates ranging from 14% to 16%. Rural areas also face higher unemployment compared to urban centers.

2. **Seasonal Variations and Urban-Rural Divide:** Unemployment rates fluctuate seasonally, with higher rates during the monsoon season due to agricultural activities and lower rates during peak tourist seasons. Urban areas have a higher unemployment rate than rural areas, partly due to industry concentration and migration from rural areas.

3. **Skill Mismatch:** There is a significant skill mismatch between the workforce and the job market, leading to high unemployment among educated individuals. This gap needs to be bridged through targeted skill development programs.

4. **Economic Growth and Infrastructure Constraints:** India's economic growth rate has slowed down, affecting job creation. Lack of adequate infrastructure in rural areas hampers economic growt

## Parallel Chain

In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

prompt1 = PromptTemplate(
    template='Generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}',
    input_variables=['notes', 'quiz']
)

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    'notes': prompt1 | model | parser,
    'quiz': prompt2 | model | parser
})

merge_chain = prompt3 | model | parser

chain = parallel_chain | merge_chain

text = """
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:

Effective in high dimensional spaces.

Still effective in cases where number of dimensions is greater than the number of samples.

Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data. For optimal performance, use C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.
"""

result = chain.invoke({'text':text})

print(result)

chain.get_graph().print_ascii()


# Support Vector Machines (SVMs)

## Purpose:
Support vector machines (SVMs) are versatile machine learning algorithms used for classification, regression, and outlier detection.

## Advantages:
- **Effective in high-dimensional spaces:** SVMs perform well even when the number of features is much larger than the number of samples.
- **Memory efficient:** They use only a subset of training points (support vectors), which makes them memory efficient.
- **Versatile with different kernel functions available:** SVMs can be adapted to various types of data and problems through the choice of kernel functions.

## Disadvantages:
- **Over-fitting risk:** When the number of features is much greater than the number of samples, there is a higher risk of over-fitting.
- **Lack of direct probability estimates:** SVMs do not provide direct probability estimates, which can be computationally expensive to obtain through cross-validation.

## Input Data:
SVMs support both dense and sparse data formats. 

## Conditional Chain

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal

parser = StrOutputParser()

class Feedback(BaseModel):
    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

parser2 = PydanticOutputParser(pydantic_object=Feedback)

prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)

classifier_chain = prompt1 | model | parser2

prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

#(condition, chain to execute when condition is true)
#(condition, chain to execute when condition is true)
#(default chain)

branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)
chain = classifier_chain | branch_chain

print(chain.invoke({'feedback': 'This is a beautiful phone'}))

chain.get_graph().print_ascii()

Thank you for your kind words! It's always great to receive positive feedback. If there's anything else I can assist you with or if you have any more questions, feel free to let me know. Your satisfaction is important to us!
    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
     +------------+      
     | ChatOllama |      
     +------------+      
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *        